# Lampung Flood Data Analysis

This notebook analyzes flood data for Jambi province using PySpark.

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# First load with pandas to check file accessibility
print("Current working directory:", os.getcwd())
print("\nListing files in current directory:")
!ls -la

# Try different possible locations of the file
possible_paths = [
    "DataFixABD.csv",
    "../data/DataFixABD.csv",
    "/home/jovyan/work/DataFixABD.csv",
    "/home/jovyan/data/DataFixABD.csv"
]

df = None
for path in possible_paths:
    try:
        print(f"\nTrying to read from: {path}")
        df = pd.read_csv(path)
        print(f"Successfully loaded data from {path}")
        break
    except Exception as e:
        print(f"Error: {str(e)}")

if df is not None:
    print("\nData sample:")
    display(df.head())
    
    print(f"\nTotal records: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
else:
    print("Failed to load the data file from any location.")

Current working directory: /home/jovyan/work

Listing files in current directory:
total 268
drwxrwxrwx 1 root   root    4096 May 26 10:10 .
drwsrws--- 1 jovyan users   4096 May 26 09:43 ..
-rwxrwxrwx 1 root   root  236685 May 25 13:24 DataFixABD.csv
-rwxrwxrwx 1 root   root   10065 May 26 10:02 flood_analysis.ipynb
-rwxrwxrwx 1 root   root    5425 May 26 10:00 hdfs_test.ipynb
drwxr-xr-x 1 jovyan users   4096 May 26 10:10 .ipynb_checkpoints
-rwxrwxrwx 1 root   root       0 May 26 10:09 spark_sql_insights.ipynb
-rwxrwxrwx 1 root   root    4277 May 26 09:59 verify_hdfs_data.ipynb

Trying to read from: DataFixABD.csv
Successfully loaded data from DataFixABD.csv

Data sample:


Tanggal Provinsi  Nilai Server  Nilai Kartu  Nilai Registered  \
0  2023-11-01    Jambi      2.192701     0.958250          1.116075   
1  2023-11-02    Jambi      2.294620     1.002791          1.167951   
2  2023-11-03    Jambi      2.161079     0.944431          1.099979   
3  2023-11-04    Jambi      2.229375     0.974277          1.134742   
4  2023-11-05    Jambi      2.053484     0.897410          1.045214   

   Nilai Unregistered  
0            1.063448  
1            1.112879  
2            1.048112  
3            1.081235  
4            0.995929


Total records: 3168
Columns: ['Tanggal', 'Provinsi', 'Nilai Server', 'Nilai Kartu', 'Nilai Registered', 'Nilai Unregistered']


In [4]:
# If data was loaded, visualize it
if df is not None:
    # Set plot style
    plt.style.use('ggplot')
    sns.set_palette("viridis")
    
    # Convert date column to datetime
    df['Tanggal'] = pd.to_datetime(df['Tanggal'])
    
    # Plot time series of flood measurements
    plt.figure(figsize=(15, 8))
    
    plt.plot(df['Tanggal'], df['Nilai_Server'], label='Server Value', linewidth=2)
    plt.plot(df['Tanggal'], df['Nilai_Kartu'], label='Card Value', linewidth=2)
    plt.plot(df['Tanggal'], df['Nilai_Registered'], label='Registered Value', linewidth=2)
    plt.plot(df['Tanggal'], df['Nilai_Unregistered'], label='Unregistered Value', linewidth=2)
    
    plt.title('Flood Measurements Over Time - Jambi Province', fontsize=16)
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Measurement Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calculate monthly averages
    df['Month'] = df['Tanggal'].dt.strftime('%Y-%m')
    monthly_avg = df.groupby('Month').mean()
    
    # Plot monthly averages
    plt.figure(figsize=(15, 8))
    monthly_avg[['Nilai_Server', 'Nilai_Kartu', 'Nilai_Registered', 'Nilai_Unregistered']].plot(kind='bar', figsize=(15, 8))
    
    plt.title('Monthly Average Flood Measurements - Jambi Province', fontsize=16)
    plt.xlabel('Month', fontsize=14)
    plt.ylabel('Average Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Cannot create visualizations because data was not loaded successfully.")

KeyError: 'Nilai_Server'

<Figure size 1500x800 with 0 Axes>

In [5]:
# Now try with PySpark for big data processing
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create Spark session
spark = SparkSession.builder \
    .appName("Flood Data Analysis") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

# Define schema for CSV
schema = StructType([
    StructField("Tanggal", StringType(), True),
    StructField("Provinsi", StringType(), True),
    StructField("Nilai_Server", DoubleType(), True),
    StructField("Nilai_Kartu", DoubleType(), True),
    StructField("Nilai_Registered", DoubleType(), True),
    StructField("Nilai_Unregistered", DoubleType(), True)
])

# Try reading with Spark, first locally
if df is not None:
    # Convert pandas to spark
    print("Converting Pandas DataFrame to Spark DataFrame...")
    spark_df = spark.createDataFrame(df)
else:
    # Try reading CSV files
    for path in possible_paths:
        try:
            print(f"\nTrying to read from: {path} with PySpark")
            spark_df = spark.read.csv(path, header=True, schema=schema)
            # Force evaluation
            count = spark_df.count()
            print(f"Successfully loaded {count} records from {path}")
            break
        except Exception as e:
            print(f"Error: {str(e)}")
    else:
        print("Failed to load the data file using PySpark.")
        spark_df = None

# Display Spark DataFrame info if loaded
if 'spark_df' in locals() and spark_df is not None:
    print("\nSpark DataFrame schema:")
    spark_df.printSchema()
    
    print("\nSample data:")
    spark_df.show(5)
    
    # Try to save to HDFS
    try:
        print("\nAttempting to save to HDFS...")
        spark_df.write.mode("overwrite").parquet("hdfs://namenode:8020/user/jovyan/flood_data_parquet")
        print("Successfully saved data to HDFS in Parquet format")
    except Exception as e:
        print(f"Error saving to HDFS: {str(e)}")

Spark version: 3.5.0
Converting Pandas DataFrame to Spark DataFrame...

Spark DataFrame schema:
root
 |-- Tanggal: timestamp (nullable = true)
 |-- Provinsi: string (nullable = true)
 |-- Nilai Server: double (nullable = true)
 |-- Nilai Kartu: double (nullable = true)
 |-- Nilai Registered: double (nullable = true)
 |-- Nilai Unregistered: double (nullable = true)


Sample data:
+-------------------+--------+------------+-----------+----------------+------------------+
|            Tanggal|Provinsi|Nilai Server|Nilai Kartu|Nilai Registered|Nilai Unregistered|
+-------------------+--------+------------+-----------+----------------+------------------+
|2023-11-01 00:00:00|   Jambi| 2.192700583| 0.95825007|     1.116074856|       1.063448421|
|2023-11-02 00:00:00|   Jambi| 2.294620126|1.002790766|      1.16795145|       1.112878871|
|2023-11-03 00:00:00|   Jambi| 2.161078688|0.944430727|     1.099979452|       1.048111966|
|2023-11-04 00:00:00|   Jambi| 2.229374932|0.974277428|     1.134

In [7]:
# If Spark data was loaded, run some analytics
if 'spark_df' in locals() and spark_df is not None:
    # Add date column
    spark_df = spark_df.withColumn("date", to_date(col("Tanggal")))
    
    # Extract month and year
    spark_df = spark_df.withColumn("month", month(col("date")))
    spark_df = spark_df.withColumn("year", year(col("date")))
    
    # Calculate daily averages
    print("\nDaily average values:")
    daily_avg = spark_df.groupBy("date").agg(
        avg("Nilai_Server").alias("avg_server"),
        avg("Nilai_Kartu").alias("avg_kartu"),
        avg("Nilai_Registered").alias("avg_registered"),
        avg("Nilai_Unregistered").alias("avg_unregistered")
    ).orderBy("date")
    
    daily_avg.show(5)
    
    # Calculate monthly averages
    print("\nMonthly average values:")
    monthly_avg = spark_df.groupBy("year", "month").agg(
        avg("Nilai_Server").alias("avg_server"),
        avg("Nilai_Kartu").alias("avg_kartu"),
        avg("Nilai_Registered").alias("avg_registered"),
        avg("Nilai_Unregistered").alias("avg_unregistered")
    ).orderBy("year", "month")
    
    monthly_avg.show()
    
    # Calculate correlation between different measurements
    print("\nCorrelation between measurements:")
    corr_server_kartu = spark_df.stat.corr("Nilai_Server", "Nilai_Kartu")
    corr_server_registered = spark_df.stat.corr("Nilai_Server", "Nilai_Registered")
    corr_server_unregistered = spark_df.stat.corr("Nilai_Server", "Nilai_Unregistered")
    
    print(f"Corr(Server, Kartu): {corr_server_kartu}")
    print(f"Corr(Server, Registered): {corr_server_registered}")
    print(f"Corr(Server, Unregistered): {corr_server_unregistered}")
    
    # Convert to Pandas for plotting
    pandas_daily = daily_avg.toPandas()
    pandas_daily['date'] = pd.to_datetime(pandas_daily['date'])
    
    # Plot using Pandas
    plt.figure(figsize=(15, 8))
    plt.plot(pandas_daily['date'], pandas_daily['avg_server'], label='Avg Server')
    plt.plot(pandas_daily['date'], pandas_daily['avg_kartu'], label='Avg Card')
    plt.plot(pandas_daily['date'], pandas_daily['avg_registered'], label='Avg Registered')
    plt.plot(pandas_daily['date'], pandas_daily['avg_unregistered'], label='Avg Unregistered')
    
    plt.title('Daily Average Flood Measurements', fontsize=16)
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Average Value', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Stop Spark session
    spark.stop()
    print("Spark session stopped.")
else:
    print("Skipping Spark analytics because data was not loaded successfully.")


Daily average values:


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Nilai_Server` cannot be resolved. Did you mean one of the following? [`Nilai Server`, `Nilai Kartu`, `Nilai Registered`, `date`, `year`].;
'Aggregate [date#87], [date#87, avg('Nilai_Server) AS avg_server#127, avg('Nilai_Kartu) AS avg_kartu#129, avg('Nilai_Registered) AS avg_registered#131, avg('Nilai_Unregistered) AS avg_unregistered#133]
+- Project [Tanggal#0, Provinsi#1, Nilai Server#2, Nilai Kartu#3, Nilai Registered#4, Nilai Unregistered#5, date#87, month#97, year(date#87) AS year#107]
   +- Project [Tanggal#0, Provinsi#1, Nilai Server#2, Nilai Kartu#3, Nilai Registered#4, Nilai Unregistered#5, date#87, month(date#87) AS month#97, year#60]
      +- Project [Tanggal#0, Provinsi#1, Nilai Server#2, Nilai Kartu#3, Nilai Registered#4, Nilai Unregistered#5, to_date(Tanggal#0, None, Some(Etc/UTC), false) AS date#87, month#51, year#60]
         +- Project [Tanggal#0, Provinsi#1, Nilai Server#2,